In [2]:
import tensorflow as tf

import numpy as np
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import functions.getLayerOutput as glo

import ssl
from tensorflow.keras import datasets, layers, models

2024-07-26 05:52:22.770059: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
glo.getMIOutput

functions.getLayerOutput.getMIOutput

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
NUM_EPOCH = 200

# training SNR
eb_n0 = 10
channel_snr = 10
rate = 1 # code rate 
channel_type = 'fading'


#trn, tst = utils.get_mnist()

# ##### KSG MI estimator #######################
# settings = {'kraskov_k': 3}
# gpu_est = est.OpenCLKraskovMI(settings = settings)
# # usage: MI = estimator.estimate(var1, var2)

##### setting for communication channel ######

def getnoisevariance(SNR,rate,P=1):
    # the SNR in args[0] is actually EbN0
    snrdB = SNR + 10*np.log10(rate)
    snr = 10.0**(snrdB/10.0)
    #P_avg = 1
    N0 = P/snr
    return (N0/2)

noise_var = getnoisevariance(eb_n0,rate,P=1)

##############################################


##############################################

label_permutation = np.arange(10)
LABEL_FIRST_HALF = label_permutation[:5]
label2_list = np.zeros((6,5))
for i in range(6):
    label2_list[i][:] = label_permutation[i:i+5]
#LABEL_SECOND_HALF = [5,6,7,8,9]

In [5]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = (x_train / 255.0)
x_test = (x_test / 255.0)
# reshape to 1d
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# save: original classification
Y_train = tf.keras.utils.to_categorical(y_train, 10)
Y_test = tf.keras.utils.to_categorical(y_test, 10)


In [6]:
x_train.shape

(60000, 784)

In [7]:
train_filter = np.where(np.in1d(y_train, LABEL_FIRST_HALF))
test_filter =  np.where(np.in1d(y_test, LABEL_FIRST_HALF))

x_1train = x_train[train_filter]
Y_1train = Y_train[train_filter]
x_1test = x_test[test_filter]
Y_1test = Y_test[test_filter]

In [8]:
### normalized Input before 
# class GaussianNoiseLayer(layers.Layer):
#     def __init__(self, stddev, **kwargs):
#         super(GaussianNoiseLayer, self).__init__(**kwargs)
#         self.stddev = stddev
    
#     def call(self, inputs, training=None):
#         if training:
#             noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.stddev)
#             return inputs + noise
#         else:
#             noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.stddev)
#             return inputs + noise
    
#     def get_config(self):
#         config = super(GaussianNoiseLayer, self).get_config()
#         config.update({'noise_var': self.stddev})
#         return config

In [9]:
## fading channel

def fading(x, stddev, h=None):
    """Implements the fading channel with multiplicative fading and
    additive white gaussian noise.
    Args:
        x: channel input symbols
        stddev: standard deviation of noise
    Returns:
        y: noisy channel output symbols
    """
    # channel gain
    if h is None:
        h = tf.complex(
            tf.random.normal([tf.shape(x)[0], 1], 0, 1 / np.sqrt(2)),
            tf.random.normal([tf.shape(x)[0], 1], 0, 1 / np.sqrt(2)),
        )

    # additive white gaussian noise
    awgn = tf.complex(
        tf.random.normal(tf.shape(x), 0, 1 / np.sqrt(2)),
        tf.random.normal(tf.shape(x), 0, 1 / np.sqrt(2)),
    )

    return (h * x + stddev * awgn), h

class Channel(layers.Layer):

#cite D-JSCC

    def __init__(self, channel_type, channel_snr, name="channel", **kwargs):
        super(Channel, self).__init__(name=name, **kwargs)
        self.channel_type = channel_type
        self.channel_snr = channel_snr

    def call(self, inputs):
        (encoded_img, prev_h) = inputs
        inter_shape = tf.shape(encoded_img)
        # reshape array to [-1, dim_z]
        z = layers.Flatten()(encoded_img)
        # convert from snr to std
        # print("channel_snr: {}".format(self.channel_snr))
        noise_stddev = np.sqrt(10 ** (-self.channel_snr / 10))

        # Add channel noise
        if self.channel_type == "awgn":
            dim_z = tf.shape(z)[1]
            # normalize latent vector so that the average power is 1
            z_in = tf.sqrt(tf.cast(dim_z, dtype=tf.float32)) * tf.nn.l2_normalize(
                z, axis=1
            )
            z_out = real_awgn(z_in, noise_stddev)
            h = tf.ones_like(z_in)  # h just makes sense on fading channels

        elif self.channel_type == "fading":
            dim_z = tf.shape(z)[1] // 2
            # convert z to complex representation
            z_in = tf.complex(z[:, :dim_z], z[:, dim_z:])
            # normalize the latent vector so that the average power is 1
            z_norm = tf.reduce_sum(
                tf.math.real(z_in * tf.math.conj(z_in)), axis=1, keepdims=True
            )
            z_in = z_in * tf.complex(
                tf.sqrt(tf.cast(dim_z, dtype=tf.float32) / z_norm), 0.0
            )
            z_out, h = fading(z_in, noise_stddev, prev_h)
            # convert back to real
            z_out = tf.concat([tf.math.real(z_out), tf.math.imag(z_out)], 1)

        # convert signal back to intermediate shape
        z_out = tf.reshape(z_out, inter_shape)

        return z_out, h


In [10]:
class CustomMetricCallback(tf.keras.callbacks.Callback):
    def __init__(self, display_frequency=10):
        super(CustomMetricCallback, self).__init__()
        self.display_frequency = display_frequency

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.display_frequency == 0:
            metrics_str = " - ".join([f"{metric_name}: {value:.4f}" for metric_name, value in logs.items()])
            print(f"Epoch {epoch}/{self.params['epochs']} - {metrics_str}")

            
callback = CustomMetricCallback(display_frequency=50)

In [16]:
SAMPLE_NUM = 1000
EPOCHS = 2


def do_report(epoch):
    if epoch == (EPOCHS-1):
        return True

    
    
def train_first_stage(lambda_val):

    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    prev_chn_gain = None

        ############ Model Structure ###############################

    input_layer  = tf.keras.layers.Input((x_train.shape[1],))
    encoder_1 = tf.keras.layers.Dense(128, activation='relu')(input_layer)
    encoder_2 =  tf.keras.layers.Dense(40, activation='relu')(encoder_1)
    ## first loss: classification 
    normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(encoder_2)
    fadingchannel = Channel(channel_type, channel_snr, name="channel_output")
    noise_layer, chn_gain = fadingchannel((normalized_x, prev_chn_gain))

    CE_decoder_1 = tf.keras.layers.Dense(40, activation='relu')(noise_layer)
    CE_decoder_2 = tf.keras.layers.Dense(40, activation='relu')(CE_decoder_1)
    CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_decoder_2)

    ## second loss: reconstruction
    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1) 
    
    #    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    #    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1)  

    model = tf.keras.Model(inputs = input_layer, outputs = [CE_output,mse_output])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-2,
    decay_steps=10000,
    decay_rate=0.8)
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    model.compile(optimizer=opt,
                  loss = {'CE' : 'categorical_crossentropy', 
                          'mse' : 'mse'},
                  metrics = {'CE' : 'accuracy', 
                             'mse': tf.keras.metrics.RootMeanSquaredError()},
                  loss_weights=[1, lambda_val])


    SNR = eb_n0
    #     print("====================")
    reporter = glo.getMIOutput(trn=[x_1train[:SAMPLE_NUM]], 
               num_selection = SAMPLE_NUM,
               embedding_network = model,
               target_layer = 2,
               do_save_func=do_report)

    history = model.fit(x=x_1train, y=(Y_1train,x_1train),
                        batch_size=128,
                        epochs=2,
                        verbose=1,
                        validation_data=(x_1test, (Y_1test,x_1test)),
                        callbacks=[reporter,])
    print("acc:", history.history['val_CE_accuracy'][-1]) 
    
    Z = reporter.datalist[-1]['data']['activity_tst'][-1] ## change 99 to -1
    current_iteration = {
    "data": Z
        }


#     ep_last = len(reporter.datalist) -1
#     # Z_data = reporter.datalist[EPOCH]['data']['activity_tst'][3][:10000]
#     ZHAT_data = reporter.datalist[ep_last]['data']['activity_tst'][4][:SAMPLE_NUM]
#     x_selected = funcs.normalize_data_new(x_1train[:SAMPLE_NUM],C=10)[0]
#     print("Current I(Z_hat; X) is: ",gpu_est.estimate(funcs.normalize_data_new(ZHAT_data,C=4)[0],x_selected))

  
    ################### secpmd stage #################
    print("stage 2!")
   
    train_filter = np.where(np.in1d(y_train, LABEL_SECOND_HALF))
    test_filter =  np.where(np.in1d(y_test, LABEL_SECOND_HALF))

    x_retrain = x_train[train_filter]
    x_retest = x_test[test_filter]
    Y_retrain = Y_train[train_filter]
    Y_retest = Y_test[test_filter]

    CE_dense_3 = layers.Dense(40, activation='relu')(noise_layer)
    CE_dense_3 = layers.Dense(40, activation='relu')(CE_dense_3)
    CE_output1 = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_3)
    #     CE_decoder_3 = tf.keras.layers.Dense(64, activation='relu')(noise_layer)
    #     CE_decoder_4 = tf.keras.layers.Dense(64, activation='relu')(CE_decoder_3)
    #     CE_output1 = tf.keras.layers.Dense(10, activation='softmax', name='CE')(CE_decoder_4)

    reconstructed_model = tf.keras.models.Model(inputs = input_layer, outputs = [CE_output1,mse_output])

    reconstructed_model.layers[1].trainable = False
    reconstructed_model.layers[2].trainable = False

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=(5e-2)*2,
    decay_steps=10000,
    decay_rate=0.8)
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)


    reconstructed_model.compile(optimizer=opt, 
                loss = {'CE' : 'categorical_crossentropy', 
                          'mse' : 'mse'
                         },              loss_weights=[1, 0],
                  metrics = {'CE' : 'accuracy', 
                             'mse': tf.keras.metrics.RootMeanSquaredError()
                           }
                 )

    #     reporter_2 = getMIOutput(trn=x_retrain, 
    #                                tst=Y_retrain, 
    #                                snr=SNR,
    #                               do_save_func=do_report)

    history1 = reconstructed_model.fit(x=x_retrain, y=(Y_retrain,x_retrain),
                        batch_size=256,
                        epochs=100,
                        verbose=0,
                        validation_data=(x_retest, (Y_retest,x_retest)))
                        # callbacks=[callback])
    print("acc:", history1.history['val_CE_accuracy'][-1]) 
    return history.history['val_CE_accuracy'][-1], history1.history['val_CE_accuracy'][-1]
    

In [18]:
Z

NameError: name 'Z' is not defined

In [17]:
sd = np.sqrt(noise_var)

lambda_list = [0,1,3,10]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0
Epoch 1/2
240/240 [==============================] - 2s 5ms/step - loss: 1.0660 - CE_loss: 1.0660 - mse_loss: 0.2331 - CE_accuracy: 0.7078 - mse_root_mean_squared_error: 0.4828 - val_loss: 0.4691 - val_CE_loss: 0.4691 - val_mse_loss: 0.2332 - val_CE_accuracy: 0.9066 - val_mse_root_mean_squared_error: 0.4830
Epoch 2/2
240/240 [==============================] - 1s 4ms/step - loss: 0.3523 - CE_loss: 0.3523 - mse_loss: 0.2331 - CE_accuracy: 0.9232 - mse_root_mean_squared_error: 0.4828 - val_loss: 0.2688 - val_CE_loss: 0.2688 - val_mse_loss: 0.2333 - val_CE_accuracy: 0.9401 - val_mse_root_mean_squared_error: 0.4830
acc: 0.9400661587715149
stage 2!


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7fe349feecb0>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [17]:
### please given data ###

lines = data.strip().split('\n')
array1 = []
array2 = []
is_stage2_next = False

for i in range(len(lines)):
    if "stage 2!" in lines[i]:
        is_stage2_next = True
        continue
    
    if lines[i].startswith("acc:"):
        acc_value = float(lines[i].split(":")[1])
        if is_stage2_next:
            array2.append(acc_value)
            is_stage2_next = False
        else:
            array1.append(acc_value)

print("Array 1:", array1)
print("Array 2:", array2)







Array 1: [0.7185999751091003, 0.7432000041007996, 0.7235999703407288, 0.7214000225067139, 0.728600025177002, 0.7211999893188477, 0.7229999899864197, 0.7287999987602234, 0.7129999995231628, 0.7084000110626221, 0.7128000259399414, 0.7134000062942505, 0.7185999751091003, 0.7369999885559082, 0.7264000177383423, 0.7347999811172485, 0.7164000272750854, 0.7103999853134155, 0.723800003528595, 0.7139999866485596, 0.7251999974250793, 0.7251999974250793, 0.7253999710083008, 0.7287999987602234, 0.7006000280380249, 0.7088000178337097, 0.715399980545044, 0.725600004196167, 0.7282000184059143, 0.7269999980926514, 0.7229999899864197, 0.7192000150680542, 0.7232000231742859, 0.7113999724388123, 0.7235999703407288, 0.728600025177002, 0.7289999723434448, 0.7354000210762024, 0.730400025844574, 0.7319999933242798, 0.7206000089645386, 0.7211999893188477, 0.7218000292778015, 0.717199981212616, 0.715399980545044, 0.7325999736785889, 0.7283999919891357, 0.7210000157356262]
Array 2: [0.8004000186920166, 0.822000

In [6]:
np.reshape(array1,(6,-1))
#print(np.reshape(array2,(6,-1)))

In [9]:
sd = np.sqrt(noise_var)

lambda_list = [0,0.2,0.5,1,3,5,10,15]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0


2023-08-11 13:31:20.039462: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78836 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-08-11 13:31:23.317045: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-08-11 13:31:23.562262: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-11 13:31:23.656432: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7faf088588c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-11 13:31:23.656474: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-08-11 13:31:23.735796: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled clus

acc: 0.7039999961853027
stage 2!
acc: 0.7901999950408936
lambda:  0.2
acc: 0.7228000164031982
stage 2!
acc: 0.8159999847412109
lambda:  0.5
acc: 0.7152000069618225
stage 2!
acc: 0.8037999868392944
lambda:  1
acc: 0.7235999703407288
stage 2!
acc: 0.8091999888420105
lambda:  3
acc: 0.7178000211715698
stage 2!
acc: 0.8068000078201294
lambda:  5
acc: 0.7300000190734863
stage 2!
acc: 0.8203999996185303
lambda:  10
acc: 0.7332000136375427
stage 2!
acc: 0.8252000212669373
lambda:  15
acc: 0.7215999960899353
stage 2!
acc: 0.8154000043869019
current overlap is:  1
lambda:  0
acc: 0.7175999879837036
stage 2!
acc: 0.7746000289916992
lambda:  0.2
acc: 0.7203999757766724
stage 2!
acc: 0.7853999733924866
lambda:  0.5
acc: 0.728600025177002
stage 2!
acc: 0.7770000100135803
lambda:  1
acc: 0.7184000015258789
stage 2!
acc: 0.7752000093460083
lambda:  3
acc: 0.7283999919891357
stage 2!
acc: 0.775600016117096
lambda:  5
acc: 0.7342000007629395
stage 2!
acc: 0.7832000255584717
lambda:  10
acc: 0.733200013

In [9]:
sd = np.sqrt(noise_var)

lambda_list = [0,0.2,0.5,1,3,5,10,15]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0


2023-08-12 04:55:05.857821: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1131 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-08-12 04:55:08.293360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-08-12 04:55:08.483354: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 891.11MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-12 04:55:08.486133: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 891.11MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-12 0

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/conv2d_1/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_once()
    F

In [10]:
sd = np.sqrt(noise_var)

lambda_list = [0,0.2,0.5,1,3,5,10,15]
acc1_list = []
acc2_list = []
for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        acc1, acc2 = train_first_stage(lambda_val)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        #train_second_stage()

current overlap is:  0
lambda:  0


2023-08-12 08:24:50.058525: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 292.97MiB (rounded to 307200000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-12 08:24:50.058600: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-08-12 08:24:50.058610: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 68, Chunks in use: 68. 17.0KiB allocated for chunks. 17.0KiB in use in bin. 1.9KiB client-requested in use in bin.
2023-08-12 08:24:50.058620: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 3, Chunks in use: 3. 1.5KiB allocated for chunks. 1.5KiB in use in bin. 1.1KiB client-requested in use in bin.
2023-08-12 08:24:50.058626: I tensorflow/tsl/framework

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

low/tsl/framework/bfc_allocator.cc:1103] 3 Chunks of size 512 totalling 1.5KiB
2023-08-12 08:24:50.059319: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 4 Chunks of size 1280 totalling 5.0KiB
2023-08-12 08:24:50.059324: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 2 Chunks of size 2048 totalling 4.0KiB
2023-08-12 08:24:50.059330: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 2560 totalling 2.5KiB
2023-08-12 08:24:50.059335: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 3 Chunks of size 10496 totalling 30.8KiB
2023-08-12 08:24:50.059340: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 20480 totalling 20.0KiB
2023-08-12 08:24:50.059346: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 26368 totalling 25.8KiB
2023-08-12 08:24:50.059351: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 32768 totalling 32.0KiB
2023-08-12 08:24:50.059356: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 8 Chunks of size

In [10]:
acc1_list

NameError: name 'acc1_list' is not defined

In [9]:
sd = np.sqrt(noise_var)

lambda_list = [0,0.2,0.5,1,3,5,10,15]
for i in range(6):
    print("current overlap is: ", (5) )
    LABEL_SECOND_HALF = label2_list[5]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        train_first_stage(lambda_val)
        #train_second_stage()

current overlap is:  5
lambda:  0


2023-08-12 08:26:59.207227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78836 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-08-12 08:27:02.458515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-08-12 08:27:02.697554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-12 08:27:02.786255: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55ef60963f20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-12 08:27:02.786299: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-08-12 08:27:02.850942: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled clus

acc: 0.7545999884605408
stage 2!
acc: 0.7680000066757202
lambda:  0.2
acc: 0.7680000066757202
stage 2!
acc: 0.7738000154495239
lambda:  0.5
acc: 0.7605999708175659
stage 2!
acc: 0.7688000202178955
lambda:  1
acc: 0.748199999332428
stage 2!
acc: 0.7626000046730042
lambda:  3
acc: 0.7738000154495239
stage 2!
acc: 0.7835999727249146
lambda:  5
acc: 0.7680000066757202
stage 2!
acc: 0.777999997138977
lambda:  10
acc: 0.7703999876976013
stage 2!
acc: 0.7788000106811523
lambda:  15
acc: 0.7767999768257141
stage 2!
acc: 0.7857999801635742
current overlap is:  5
lambda:  0
acc: 0.7581999897956848
stage 2!
acc: 0.7694000005722046
lambda:  0.2
acc: 0.7487999796867371
stage 2!
acc: 0.7656000256538391
lambda:  0.5
acc: 0.7563999891281128
stage 2!
acc: 0.7645999789237976
lambda:  1
acc: 0.7572000026702881
stage 2!
acc: 0.7742000222206116
lambda:  3
acc: 0.7752000093460083
stage 2!
acc: 0.7918000221252441
lambda:  5
acc: 0.769599974155426
stage 2!
acc: 0.7666000127792358
lambda:  10
acc: 0.7688000202

## MI estiamtion

In [23]:
SAMPLE_NUM = 10000
EPOCHS = 100


def do_report(epoch):
    if epoch == (EPOCHS-1):
        return True

    
    
def train_first_stage(lambda_val):

    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    prev_chn_gain = None

        ############ Model Structure ###############################

    input_layer  = tf.keras.layers.Input((x_train.shape[1],))
    encoder_1 = tf.keras.layers.Dense(128, activation='relu')(input_layer)
    encoder_2 =  tf.keras.layers.Dense(40, activation='relu')(encoder_1)
    ## first loss: classification 
    normalized_x = tf.keras.layers.Lambda(lambda x: K.tanh(x))(encoder_2)
    fadingchannel = Channel(channel_type, channel_snr, name="channel_output")
    noise_layer, chn_gain = fadingchannel((normalized_x, prev_chn_gain))

    CE_decoder_1 = tf.keras.layers.Dense(40, activation='relu')(noise_layer)
    CE_decoder_2 = tf.keras.layers.Dense(40, activation='relu')(CE_decoder_1)
    CE_output = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_decoder_2)

    ## second loss: reconstruction
    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1) 
    
    #    mse_decoder_1 = tf.keras.layers.Dense(256, activation='relu')(noise_layer)
    #    mse_output = tf.keras.layers.Dense(x_train.shape[1],activation = 'sigmoid',name='mse')(mse_decoder_1)  

    model = tf.keras.Model(inputs = input_layer, outputs = [CE_output,mse_output])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-2,
    decay_steps=10000,
    decay_rate=0.8)
    opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
    model.compile(optimizer=opt,
                  loss = {'CE' : 'categorical_crossentropy', 
                          'mse' : 'mse'},
                  metrics = {'CE' : 'accuracy', 
                             'mse': tf.keras.metrics.RootMeanSquaredError()},
                  loss_weights=[1, lambda_val])


    SNR = eb_n0
    #     print("====================")
    reporter = glo.getMIOutput(trn=[x_1train[:SAMPLE_NUM]], 
               num_selection = SAMPLE_NUM,
               embedding_network = model,
               target_layer = 2,
               do_save_func=do_report)

    history = model.fit(x=x_1train, y=(Y_1train,x_1train),
                        batch_size=128,
                        epochs=EPOCHS,
                        verbose=1,
                        validation_data=(x_1test, (Y_1test,x_1test)),
                        callbacks=[reporter,])
    print("acc:", history.history['val_CE_accuracy'][-1]) 
    
    Z = reporter.datalist[-1]['data']['activity_tst'][-1] ## change 99 to -1
#     current_iteration = {
#     "data": Z
#         }
    return Z

#     ep_last = len(reporter.datalist) -1
#     # Z_data = reporter.datalist[EPOCH]['data']['activity_tst'][3][:10000]
#     ZHAT_data = reporter.datalist[ep_last]['data']['activity_tst'][4][:SAMPLE_NUM]
#     x_selected = funcs.normalize_data_new(x_1train[:SAMPLE_NUM],C=10)[0]
#     print("Current I(Z_hat; X) is: ",gpu_est.estimate(funcs.normalize_data_new(ZHAT_data,C=4)[0],x_selected))

  
#     ################### secpmd stage #################
#     print("stage 2!")
   
#     train_filter = np.where(np.in1d(y_train, LABEL_SECOND_HALF))
#     test_filter =  np.where(np.in1d(y_test, LABEL_SECOND_HALF))

#     x_retrain = x_train[train_filter]
#     x_retest = x_test[test_filter]
#     Y_retrain = Y_train[train_filter]
#     Y_retest = Y_test[test_filter]

#     CE_dense_3 = layers.Dense(40, activation='relu')(noise_layer)
#     CE_dense_3 = layers.Dense(40, activation='relu')(CE_dense_3)
#     CE_output1 = tf.keras.layers.Dense(10,activation = 'softmax',name='CE')(CE_dense_3)
#     #     CE_decoder_3 = tf.keras.layers.Dense(64, activation='relu')(noise_layer)
#     #     CE_decoder_4 = tf.keras.layers.Dense(64, activation='relu')(CE_decoder_3)
#     #     CE_output1 = tf.keras.layers.Dense(10, activation='softmax', name='CE')(CE_decoder_4)

#     reconstructed_model = tf.keras.models.Model(inputs = input_layer, outputs = [CE_output1,mse_output])

#     reconstructed_model.layers[1].trainable = False
#     reconstructed_model.layers[2].trainable = False

#     lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=(5e-2)*2,
#     decay_steps=10000,
#     decay_rate=0.8)
#     opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)


#     reconstructed_model.compile(optimizer=opt, 
#                 loss = {'CE' : 'categorical_crossentropy', 
#                           'mse' : 'mse'
#                          },              loss_weights=[1, 0],
#                   metrics = {'CE' : 'accuracy', 
#                              'mse': tf.keras.metrics.RootMeanSquaredError()
#                            }
#                  )

#     #     reporter_2 = getMIOutput(trn=x_retrain, 
#     #                                tst=Y_retrain, 
#     #                                snr=SNR,
#     #                               do_save_func=do_report)

#     history1 = reconstructed_model.fit(x=x_retrain, y=(Y_retrain,x_retrain),
#                         batch_size=256,
#                         epochs=100,
#                         verbose=0,
#                         validation_data=(x_retest, (Y_retest,x_retest)))
#                         # callbacks=[callback])
#     print("acc:", history1.history['val_CE_accuracy'][-1]) 
#     return history.history['val_CE_accuracy'][-1], history1.history['val_CE_accuracy'][-1]
    

In [24]:
sd = np.sqrt(noise_var)

lambda_list = [0,1,3,10]
z_list = []

for i in range(6):
    print("current overlap is: ", (i) )
    LABEL_SECOND_HALF = label2_list[5-i]
    for lambda_val in lambda_list:
        print("lambda: ",lambda_val)
        z = train_first_stage(lambda_val)
        z_list.append(z)

        #train_second_stage()

current overlap is:  0
lambda:  0
Epoch 1/100
240/240 [==============================] - 2s 6ms/step - loss: 1.0164 - CE_loss: 1.0164 - mse_loss: 0.2344 - CE_accuracy: 0.7107 - mse_root_mean_squared_error: 0.4842 - val_loss: 0.4514 - val_CE_loss: 0.4514 - val_mse_loss: 0.2346 - val_CE_accuracy: 0.9062 - val_mse_root_mean_squared_error: 0.4844
Epoch 2/100
240/240 [==============================] - 1s 4ms/step - loss: 0.3418 - CE_loss: 0.3418 - mse_loss: 0.2344 - CE_accuracy: 0.9209 - mse_root_mean_squared_error: 0.4842 - val_loss: 0.2465 - val_CE_loss: 0.2465 - val_mse_loss: 0.2346 - val_CE_accuracy: 0.9461 - val_mse_root_mean_squared_error: 0.4844
Epoch 3/100
240/240 [==============================] - 1s 4ms/step - loss: 0.2445 - CE_loss: 0.2445 - mse_loss: 0.2344 - CE_accuracy: 0.9400 - mse_root_mean_squared_error: 0.4842 - val_loss: 0.1918 - val_CE_loss: 0.1918 - val_mse_loss: 0.2346 - val_CE_accuracy: 0.9525 - val_mse_root_mean_squared_error: 0.4844
Epoch 4/100
240/240 [============

240/240 [==============================] - 1s 4ms/step - loss: 0.0419 - CE_loss: 0.0419 - mse_loss: 0.2344 - CE_accuracy: 0.9853 - mse_root_mean_squared_error: 0.4842 - val_loss: 0.1183 - val_CE_loss: 0.1183 - val_mse_loss: 0.2347 - val_CE_accuracy: 0.9768 - val_mse_root_mean_squared_error: 0.4844
Epoch 54/100
240/240 [==============================] - 1s 4ms/step - loss: 0.0377 - CE_loss: 0.0377 - mse_loss: 0.2345 - CE_accuracy: 0.9868 - mse_root_mean_squared_error: 0.4842 - val_loss: 0.1034 - val_CE_loss: 0.1034 - val_mse_loss: 0.2346 - val_CE_accuracy: 0.9772 - val_mse_root_mean_squared_error: 0.4844
Epoch 55/100
240/240 [==============================] - 1s 4ms/step - loss: 0.0401 - CE_loss: 0.0401 - mse_loss: 0.2344 - CE_accuracy: 0.9859 - mse_root_mean_squared_error: 0.4842 - val_loss: 0.1153 - val_CE_loss: 0.1153 - val_mse_loss: 0.2346 - val_CE_accuracy: 0.9768 - val_mse_root_mean_squared_error: 0.4844
Epoch 56/100
240/240 [==============================] - 1s 4ms/step - loss: 0

240/240 [==============================] - 1s 4ms/step - loss: 0.3793 - CE_loss: 0.1619 - mse_loss: 0.2174 - CE_accuracy: 0.9575 - mse_root_mean_squared_error: 0.4663 - val_loss: 0.3620 - val_CE_loss: 0.1464 - val_mse_loss: 0.2156 - val_CE_accuracy: 0.9646 - val_mse_root_mean_squared_error: 0.4643
Epoch 6/100
240/240 [==============================] - 1s 4ms/step - loss: 0.3534 - CE_loss: 0.1396 - mse_loss: 0.2137 - CE_accuracy: 0.9631 - mse_root_mean_squared_error: 0.4623 - val_loss: 0.3500 - val_CE_loss: 0.1380 - val_mse_loss: 0.2119 - val_CE_accuracy: 0.9646 - val_mse_root_mean_squared_error: 0.4604
Epoch 7/100
240/240 [==============================] - 1s 4ms/step - loss: 0.3380 - CE_loss: 0.1280 - mse_loss: 0.2100 - CE_accuracy: 0.9647 - mse_root_mean_squared_error: 0.4582 - val_loss: 0.3277 - val_CE_loss: 0.1195 - val_mse_loss: 0.2082 - val_CE_accuracy: 0.9661 - val_mse_root_mean_squared_error: 0.4562
Epoch 8/100
240/240 [==============================] - 1s 4ms/step - loss: 0.31

240/240 [==============================] - 1s 4ms/step - loss: 0.1213 - CE_loss: 0.0411 - mse_loss: 0.0802 - CE_accuracy: 0.9854 - mse_root_mean_squared_error: 0.2832 - val_loss: 0.1720 - val_CE_loss: 0.0922 - val_mse_loss: 0.0798 - val_CE_accuracy: 0.9802 - val_mse_root_mean_squared_error: 0.2825
Epoch 58/100
240/240 [==============================] - 1s 4ms/step - loss: 0.1183 - CE_loss: 0.0386 - mse_loss: 0.0797 - CE_accuracy: 0.9864 - mse_root_mean_squared_error: 0.2823 - val_loss: 0.1642 - val_CE_loss: 0.0849 - val_mse_loss: 0.0793 - val_CE_accuracy: 0.9833 - val_mse_root_mean_squared_error: 0.2817
Epoch 59/100
240/240 [==============================] - 1s 4ms/step - loss: 0.1193 - CE_loss: 0.0400 - mse_loss: 0.0793 - CE_accuracy: 0.9858 - mse_root_mean_squared_error: 0.2817 - val_loss: 0.1757 - val_CE_loss: 0.0966 - val_mse_loss: 0.0791 - val_CE_accuracy: 0.9827 - val_mse_root_mean_squared_error: 0.2813
Epoch 60/100
240/240 [==============================] - 1s 4ms/step - loss: 0

240/240 [==============================] - 1s 4ms/step - loss: 0.4637 - CE_loss: 0.1097 - mse_loss: 0.1180 - CE_accuracy: 0.9702 - mse_root_mean_squared_error: 0.3435 - val_loss: 0.4592 - val_CE_loss: 0.1215 - val_mse_loss: 0.1125 - val_CE_accuracy: 0.9691 - val_mse_root_mean_squared_error: 0.3355
Epoch 10/100
240/240 [==============================] - 1s 4ms/step - loss: 0.4231 - CE_loss: 0.0990 - mse_loss: 0.1080 - CE_accuracy: 0.9708 - mse_root_mean_squared_error: 0.3287 - val_loss: 0.4203 - val_CE_loss: 0.1092 - val_mse_loss: 0.1037 - val_CE_accuracy: 0.9714 - val_mse_root_mean_squared_error: 0.3220
Epoch 11/100
240/240 [==============================] - 1s 4ms/step - loss: 0.3961 - CE_loss: 0.0946 - mse_loss: 0.1005 - CE_accuracy: 0.9731 - mse_root_mean_squared_error: 0.3170 - val_loss: 0.4267 - val_CE_loss: 0.1346 - val_mse_loss: 0.0974 - val_CE_accuracy: 0.9632 - val_mse_root_mean_squared_error: 0.3121
Epoch 12/100
240/240 [==============================] - 1s 4ms/step - loss: 0

...

240/240 [==============================] - 1s 4ms/step - loss: 0.1094 - CE_loss: 0.0358 - mse_loss: 0.0736 - CE_accuracy: 0.9878 - mse_root_mean_squared_error: 0.2712 - val_loss: 0.1917 - val_CE_loss: 0.1184 - val_mse_loss: 0.0734 - val_CE_accuracy: 0.9774 - val_mse_root_mean_squared_error: 0.2709
Epoch 86/100
240/240 [==============================] - 1s 4ms/step - loss: 0.1127 - CE_loss: 0.0393 - mse_loss: 0.0734 - CE_accuracy: 0.9866 - mse_root_mean_squared_error: 0.2709 - val_loss: 0.1932 - val_CE_loss: 0.1200 - val_mse_loss: 0.0732 - val_CE_accuracy: 0.9800 - val_mse_root_mean_squared_error: 0.2706
Epoch 87/100
240/240 [==============================] - 1s 4ms/step - loss: 0.1106 - CE_loss: 0.0372 - mse_loss: 0.0733 - CE_accuracy: 0.9869 - mse_root_mean_squared_error: 0.2708 - val_loss: 0.1724 - val_CE_loss: 0.0993 - val_mse_loss: 0.0731 - val_CE_accuracy: 0.9813 - val_mse_root_mean_squared_error: 0.2703
Epoch 88/100
240/240 [==============================] - 1s 4ms/step - loss: 0

240/240 [==============================] - 1s 4ms/step - loss: 0.2695 - CE_loss: 0.0594 - mse_loss: 0.0700 - CE_accuracy: 0.9819 - mse_root_mean_squared_error: 0.2646 - val_loss: 0.3140 - val_CE_loss: 0.1043 - val_mse_loss: 0.0699 - val_CE_accuracy: 0.9698 - val_mse_root_mean_squared_error: 0.2644
Epoch 38/100
240/240 [==============================] - 1s 4ms/step - loss: 0.2638 - CE_loss: 0.0541 - mse_loss: 0.0699 - CE_accuracy: 0.9818 - mse_root_mean_squared_error: 0.2644 - val_loss: 0.2999 - val_CE_loss: 0.0911 - val_mse_loss: 0.0696 - val_CE_accuracy: 0.9800 - val_mse_root_mean_squared_error: 0.2638
Epoch 39/100
240/240 [==============================] - 1s 4ms/step - loss: 0.2557 - CE_loss: 0.0465 - mse_loss: 0.0697 - CE_accuracy: 0.9843 - mse_root_mean_squared_error: 0.2641 - val_loss: 0.3031 - val_CE_loss: 0.0944 - val_mse_loss: 0.0695 - val_CE_accuracy: 0.9755 - val_mse_root_mean_squared_error: 0.2637
Epoch 40/100
240/240 [==============================] - 1s 4ms/step - loss: 0

240/240 [==============================] - 1s 4ms/step - loss: 0.2337 - CE_loss: 0.0373 - mse_loss: 0.0655 - CE_accuracy: 0.9864 - mse_root_mean_squared_error: 0.2559 - val_loss: 0.3143 - val_CE_loss: 0.1185 - val_mse_loss: 0.0653 - val_CE_accuracy: 0.9803 - val_mse_root_mean_squared_error: 0.2555
Epoch 90/100
240/240 [==============================] - 1s 4ms/step - loss: 0.2355 - CE_loss: 0.0393 - mse_loss: 0.0654 - CE_accuracy: 0.9864 - mse_root_mean_squared_error: 0.2557 - val_loss: 0.3066 - val_CE_loss: 0.1114 - val_mse_loss: 0.0651 - val_CE_accuracy: 0.9780 - val_mse_root_mean_squared_error: 0.2551
Epoch 91/100
240/240 [==============================] - 1s 4ms/step - loss: 0.2315 - CE_loss: 0.0354 - mse_loss: 0.0653 - CE_accuracy: 0.9879 - mse_root_mean_squared_error: 0.2556 - val_loss: 0.3120 - val_CE_loss: 0.1167 - val_mse_loss: 0.0651 - val_CE_accuracy: 0.9788 - val_mse_root_mean_squared_error: 0.2551
Epoch 92/100
240/240 [==============================] - 1s 4ms/step - loss: 0

240/240 [==============================] - 1s 4ms/step - loss: 0.6573 - CE_loss: 0.0491 - mse_loss: 0.0608 - CE_accuracy: 0.9835 - mse_root_mean_squared_error: 0.2466 - val_loss: 0.7174 - val_CE_loss: 0.1132 - val_mse_loss: 0.0604 - val_CE_accuracy: 0.9747 - val_mse_root_mean_squared_error: 0.2458
Epoch 42/100
240/240 [==============================] - 1s 4ms/step - loss: 0.6481 - CE_loss: 0.0421 - mse_loss: 0.0606 - CE_accuracy: 0.9853 - mse_root_mean_squared_error: 0.2462 - val_loss: 0.6993 - val_CE_loss: 0.0981 - val_mse_loss: 0.0601 - val_CE_accuracy: 0.9780 - val_mse_root_mean_squared_error: 0.2452
Epoch 43/100
240/240 [==============================] - 1s 4ms/step - loss: 0.6500 - CE_loss: 0.0453 - mse_loss: 0.0605 - CE_accuracy: 0.9852 - mse_root_mean_squared_error: 0.2459 - val_loss: 0.7162 - val_CE_loss: 0.1154 - val_mse_loss: 0.0601 - val_CE_accuracy: 0.9759 - val_mse_root_mean_squared_error: 0.2451
Epoch 44/100
240/240 [==============================] - 1s 4ms/step - loss: 0

240/240 [==============================] - 1s 4ms/step - loss: 0.5918 - CE_loss: 0.0408 - mse_loss: 0.0551 - CE_accuracy: 0.9857 - mse_root_mean_squared_error: 0.2347 - val_loss: 0.6460 - val_CE_loss: 0.1021 - val_mse_loss: 0.0544 - val_CE_accuracy: 0.9792 - val_mse_root_mean_squared_error: 0.2332
Epoch 94/100
240/240 [==============================] - 1s 4ms/step - loss: 0.5941 - CE_loss: 0.0434 - mse_loss: 0.0551 - CE_accuracy: 0.9849 - mse_root_mean_squared_error: 0.2347 - val_loss: 0.6431 - val_CE_loss: 0.0984 - val_mse_loss: 0.0545 - val_CE_accuracy: 0.9802 - val_mse_root_mean_squared_error: 0.2334
Epoch 95/100
240/240 [==============================] - 1s 4ms/step - loss: 0.5888 - CE_loss: 0.0398 - mse_loss: 0.0549 - CE_accuracy: 0.9862 - mse_root_mean_squared_error: 0.2343 - val_loss: 0.6472 - val_CE_loss: 0.1027 - val_mse_loss: 0.0545 - val_CE_accuracy: 0.9747 - val_mse_root_mean_squared_error: 0.2333
Epoch 96/100
240/240 [==============================] - 1s 4ms/step - loss: 0

In [49]:
from functions.MIestimate import MINE_MI
from functions.MIestimate import variance_normalization as VN

In [48]:
x_selected = VN(x_1train[:SAMPLE_NUM],C=1)[0]

In [46]:
MINE_MI(A,A,total_epochs=60)

(41.12915839552861, <keras.callbacks.History at 0x7fe0f479a200>)

In [53]:

##### TEST THE RANGE FOR PHASE 2

izx_val_C_z=[]
se = []

count = 0
#dnew=z_selected
for i in range(6):
    print("current overlap is: ", (i) )
    for lambda_val in lambda_list:
        print("lambda: ", lambda_val)
        z = z_list[count]
        dnew = VN(z,C=1)[0]

        #dnew = z[:SAMPLE_NUM]
        normz,_ = VN(dnew,C=1)
        izx_val,_ = MINE_MI(dnew,x_selected,total_epochs=50)

        print("I_Z_X: ",izx_val)
        #print("I_Z_Y: ",izy_val)
        izx_val_C_z.append(izx_val)
        count+=1



current overlap is:  0
lambda:  0


I_Z_X:  2.9754364141513743
lambda:  1


I_Z_X:  2.351599445110437
lambda:  3


I_Z_X:  2.241639069610934
lambda:  10


I_Z_X:  2.882739485162829
current overlap is:  1
lambda:  0


...

I_Z_X:  2.4403966674906683
lambda:  10


I_Z_X:  3.0021721491239033
current overlap is:  5
lambda:  0


I_Z_X:  2.725899906708491
lambda:  1


I_Z_X:  2.4523130014036503
lambda:  3


I_Z_X:  2.3812053985432153
lambda:  10
I_Z_X:  3.0786511141360857


In [1]:
array([2.8692471 , 2.3776179 , 2.40168653, 3.04009828])

NameError: name 'np' is not defined

In [75]:
# Save the entire list using pickle
import pickle
with open('z_mnist_fading.pkl', 'wb') as f:
    pickle.dump(z_list, f)

In [65]:
def global_normalize_mine(data_orig, C):
    # Reshape data as before
    data = np.reshape(data_orig, (data_orig.shape[0], int(data_orig.size / data_orig.shape[0])))
    
    # Subtract the mean
    means = np.mean(data, axis=0)
    data = np.abs(data - means)
    
    # Compute the normalization factor
    # norm = np.sqrt(np.mean(np.sum(data ** 2, axis=1)))
    # linalg.norm(x, ord=None, 
    norm = np.sqrt(np.mean(np.sum(data ** 2, axis=1)))
    norm = norm/ np.sqrt(data.shape[1])
    
    # print(norm)
    # Normalize the data
    
    data = C*data_orig / norm
    return data

In [74]:

##### TEST THE RANGE FOR PHASE 2

izx_val_C_z=[]
se = []

count = 0
#dnew=z_selected
for i in range(6):
    print("current overlap is: ", (i) )
    for lambda_val in lambda_list:
        print("lambda: ", lambda_val)
        z = z_list[count]
        dnew = VN(z,C=1)[0]

        #dnew = z[:SAMPLE_NUM]
        
        #normz,_ = VN(dnew,C=1)
        izx_val,_ = MINE_MI(dnew,x_selected,total_epochs=50)

        print("I_Z_X: ",izx_val)
        #print("I_Z_Y: ",izy_val)
        izx_val_C_z.append(izx_val)
        count+=1



current overlap is:  0
lambda:  0


I_Z_X:  1.148905161604618
lambda:  1


I_Z_X:  0.6850456374786736
lambda:  3


I_Z_X:  1.5967646667685054
lambda:  10


I_Z_X:  2.3628347277260024
current overlap is:  1
lambda:  0


...

I_Z_X:  1.4207324264171555
lambda:  10


I_Z_X:  2.4810889665633646
current overlap is:  5
lambda:  0


I_Z_X:  1.329271278766385
lambda:  1


I_Z_X:  0.5832660467240872
lambda:  3


I_Z_X:  1.6243355494936238
lambda:  10
I_Z_X:  2.482817048239234


In [76]:
np.reshape(izx_val_C_z,(6,-1))

array([[1.14890516, 0.68504564, 1.59676467, 2.36283473],
       [1.11080094, 0.6105913 , 1.57846313, 2.57590747],
       [1.1022909 , 0.78434554, 1.51329719, 2.58763703],
       [0.86338765, 0.33282115, 1.5714655 , 2.18869473],
       [1.34816022, 0.3386051 , 1.42073243, 2.48108897],
       [1.32927128, 0.58326605, 1.62433555, 2.48281705]])

In [77]:
np.mean(np.reshape(izx_val_C_z,(6,-1)),axis=0)

array([1.15046936, 0.55577913, 1.55084308, 2.44649666])